In [1]:
import lightgbm as lgb
import sys
from os import listdir
import os.path
from os.path import isdir, isfile, join
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
def readCSVfromDirectory(dir):
    print(dir)
    filepaths = []
    for f in listdir(dir):
        path = join(dir, f)
        if isfile(path) and f.endswith(".csv"):
            filepaths.append(path)

    if len(filepaths) == 0:
        exit("Directory does not contain any csv files: %s" % dir)

    filepaths = np.sort(filepaths)
    print("Found %s files." % len(filepaths))

    return filepaths


In [3]:
#############################################
#MARK: - Read Files

SOURCE_DIR = "./data/"


SOURCE_DIR = os.path.abspath(SOURCE_DIR)
if not isdir(SOURCE_DIR):
    exit("That's not a directory!")
else:
    print(SOURCE_DIR)

elems = listdir(SOURCE_DIR)
print(elems)
directories = {}
for e in elems:
    path = join(SOURCE_DIR, e)
    
    if isdir(path):
        filepaths = readCSVfromDirectory(path)[:25]

        directories[e] = []
        for filepath in filepaths:
            # the files contain values in [V] and [A]
            # use a current Factor of 10e-3 if given in [mA]
            dataframe = pd.read_csv(filepath, header=1)
            # Felix ist schuld
            dataframe.columns = dataframe.columns.str.strip().str.replace('(', '').str.replace(')', '') 
            #create id field
            dataframe = dataframe.assign(id= lambda x: dataframe.index )
            #remove ms field
            dataframe = dataframe.drop(columns=['ms'])
            if 'mA' in dataframe:
                dataframe = dataframe.assign(A= lambda x: x.mA / 1000)
                dataframe = dataframe.drop(columns=['mA'])
                #print("Converted mA to A")
            directories[e].append(dataframe)
            

/Users/paul/DataScienceProjekt/data
['ohp_voll-single.psdata', 'laptop-single', 'ohp_voll-laptop', 'lampe-ohp_voll.psdata', 'ohp_halb-single', 'ohp_halb-single.psdata', 'ohp_voll-laptop-bosch-lampe.psdata', 'lampe-ohp_voll', 'ohp_voll-single', 'ohp_halb-laptop-lampe.psdata', 'ohp_halb-laptop.psdata', 'bosch-single', 'lampe-single', 'ohp_voll-laptop-bosch-lampe', 'ohp_halb-laptop', 'ohp_halb-laptop-lampe']
/Users/paul/DataScienceProjekt/data/laptop-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-laptop
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_halb-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/lampe-ohp_voll
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/bosch-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/lampe-single
Found 256 files.
/Users/paul/DataScienceProjekt/data/ohp_voll-laptop-bosch-lampe
Found 256 files.
/Users/paul/DataScienceProjekt/data/

In [4]:
for (id, dfs) in directories.items():
    print(id, len(dfs))
    print(dfs[0].head(4))
    print(dfs[0].describe())
    print('\n\n')

laptop-single 25
          V         A  id
0 -8.224738  0.018067   0
1 -8.224738  0.036012   1
2 -3.753777  0.018067   2
3  0.732444  0.018067   3
                 V            A           id
count  2445.000000  2445.000000  2445.000000
mean     38.888648     0.021008  1222.000000
std     222.962687     0.338291   705.955027
min    -321.848200    -1.293191     0.000000
25%    -178.472200    -0.017823   611.000000
50%      81.362350     0.000122  1222.000000
75%     251.609800     0.018067  1833.000000
max     314.340600     1.419172  2444.000000



ohp_voll-laptop 25
           V         A  id
0 -12.710960 -1.078524   0
1  -8.224738 -1.078524   1
2  -3.753777 -1.078524   2
3  -8.224738 -1.078524   3
                 V            A           id
count  2445.000000  2445.000000  2445.000000
mean     38.879487     0.443960  1222.000000
std     221.681810     2.560982   705.955027
min    -317.362000    -5.035249     0.000000
25%    -174.001300    -1.887875   611.000000
50%      81.362350   

#### TODO

- Zeit diskretisieren 0ms = 0 und 20ms = 2445 (neue spalte -> id des datensatzes) done
- mA zu A umrechnen (falls vorhanden) done
- geht multilabel mit gradient boosting? jaein, vielleicht
- in X und Y aufteilen 
- attacke!


In [5]:
features = []

for label, data in directories.items():
    # pro gerät alle daten in einen dataframe
    res = pd.concat(data, sort=False)
    #zu der tabelle eine spalte mit label hinzufügen
    res = res.assign(labels= lambda x: label)
    
    features.append(res)

# dataframes zu einem kombinieren
features = pd.concat(features, sort=False)

X = features.drop(columns=['labels'])
min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)
Y = features['labels']

print(X)

Y = pd.get_dummies(Y)
Y = Y.assign(labels= lambda x: 0 + x['lampe-ohp_voll'] + x['lampe-single'] * 2 + x['laptop-single'] * 3 + x['ohp_halb-laptop']* 4 + x['ohp_halb-laptop-lampe'] * 5 + x['ohp_halb-single'] * 6 + x['ohp_voll-laptop'] * 7 + x['ohp_voll-laptop-bosch-lampe'] * 8 + x['ohp_voll-single'] *9)
Y = Y['labels']


[[4.92972309e-01 4.97742477e-01 0.00000000e+00]
 [4.92972309e-01 4.99220681e-01 4.09165303e-04]
 [5.00000036e-01 4.97742477e-01 8.18330606e-04]
 ...
 [5.07051750e-01 5.62949378e-01 9.99181669e-01]
 [5.07051750e-01 5.62949378e-01 9.99590835e-01]
 [5.00000036e-01 5.55533218e-01 1.00000000e+00]]


In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, shuffle=True)

In [12]:
#train_data = lgb.Dataset(X_train, label=Y_train)
#test_data = lgb.Dataset(X_test, label=Y_test)

#param = {'num_leaves':200, 'num_trees':100, 'objective':'multiclass', 'num_class': 10}
#num_round = 100
#param['metric'] = ['multi_logloss', 'multi_error']

#bst = lgb.cv(param, train_data, num_round, nfold=5)
#bst = lgb.train(param, train_data, num_round, valid_sets=[train_data, test_data], valid_names = ['train','eval'])

/Users/paul/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	train's multi_error: 0.253337	train's multi_logloss: 1.99092	eval's multi_error: 0.257006	eval's multi_logloss: 1.9917
[2]	train's multi_error: 0.228667	train's multi_logloss: 1.78352	eval's multi_error: 0.232982	eval's multi_logloss: 1.78445
[3]	train's multi_error: 0.212493	train's multi_logloss: 1.62434	eval's multi_error: 0.215419	eval's multi_logloss: 1.6255
[4]	train's multi_error: 0.203151	train's multi_logloss: 1.49454	eval's multi_error: 0.206176	eval's multi_logloss: 1.49611
[5]	train's multi_error: 0.198358	train's multi_logloss: 1.38556	eval's multi_error: 0.201628	eval's multi_logloss: 1.38743
[6]	train's multi_error: 0.193456	train's multi_logloss: 1.29166	eval's multi_error: 0.196434	eval's multi_logloss: 1.29373
[7]	train's multi_error: 0.189793	train's multi_logloss: 1.21058	eval's multi_error: 0.192826	eval's multi_logloss: 1.2129
[8]	train's multi_error: 0.185585	train's multi_logloss: 1.13958	eval's multi_error: 0.188352	eval's multi_logloss: 1.14204
[9]	train's

[66]	train's multi_error: 0.146699	train's multi_logloss: 0.344019	eval's multi_error: 0.150822	eval's multi_logloss: 0.351549
[67]	train's multi_error: 0.146493	train's multi_logloss: 0.342414	eval's multi_error: 0.150658	eval's multi_logloss: 0.349999
[68]	train's multi_error: 0.146427	train's multi_logloss: 0.340675	eval's multi_error: 0.150806	eval's multi_logloss: 0.34833
[69]	train's multi_error: 0.146384	train's multi_logloss: 0.339153	eval's multi_error: 0.150511	eval's multi_logloss: 0.346868
[70]	train's multi_error: 0.146342	train's multi_logloss: 0.337697	eval's multi_error: 0.150691	eval's multi_logloss: 0.345497
[71]	train's multi_error: 0.146162	train's multi_logloss: 0.336283	eval's multi_error: 0.150601	eval's multi_logloss: 0.344139
[72]	train's multi_error: 0.146025	train's multi_logloss: 0.334969	eval's multi_error: 0.150421	eval's multi_logloss: 0.342922
[73]	train's multi_error: 0.145957	train's multi_logloss: 0.333779	eval's multi_error: 0.150331	eval's multi_log

In [49]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=50, random_state=42, min_samples_split=2)
clf.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [50]:
print("Train:", round(10000*clf.score(X_train, Y_train))/100.0)
print("Test:", round(10000*clf.score(X_test, Y_test))/100.0)

Train: 90.39
Test: 89.07
